## 爬取Pokemon图片

In [12]:
import requests
from bs4 import BeautifulSoup
import re
import os
from IPython.display import Image
import time

# url 前缀
url_prefix = 'https://wiki.52poke.com'

**1.获取网页源码**

In [13]:
def crawl(url, url_suffix):
    '''
    根据传入的 url 获取html
    returns:
        html: 获取到的 html
    '''
    url = url + url_suffix
    html = requests.get(url).text
    return html

**2.解析网页**

In [14]:
def parse(html):
    '''
    根据传入的 html 找到我们需要的数据
    return:
        all_pokemon_urls: 所有 Pokemon 的url
    '''
    # 创建 html 实例
    soup = BeautifulSoup(html, 'html.parser')
    
    # 所以所有 href 标签内容
    all_hrefs = soup.find(class_='mw-content-ltr').find_all(class_='mw-redirect')
    
    # 提取所有链接
    all_urls = [url.get('href') for url in all_hrefs]

    # 仅提取格式为 `/wiki/Bulbasaur` 的所有链接
    all_pokemon_urls = []
    # 使用正则表达式匹配想要的链接
    # 但是执行速度比使用下面的while慢了一倍
#     for i in range(len(all_urls)):
#         url = all_urls[i]
#         find = re.search(r'/wiki/\w.*', url)
#         if find:
#             all_pokemon_urls.append(find.group(0)
    i = 0
    while i < len(all_urls):
        if i%2 != 0:
            all_pokemon_urls.append(all_urls[i])
        i += 1
    
    return all_pokemon_urls

**3.获取下一层级：Pokemon 详细信息页面的链接**

In [15]:
now = time.time()

# 获取 Pokemon html
html = crawl(url_prefix, '/wiki/宝可梦列表（按全国图鉴编号）/简单版')
# 获取所有 Pokemon 详细信息链接
all_pokemon_urls = parse(html)

print('爬取完成，共花费 {} 秒'.format(time.time()-now))

爬取完成，共花费 1.771186113357544 秒


In [16]:
all_pokemon_urls

['/wiki/Bulbasaur',
 '/wiki/Ivysaur',
 '/wiki/Venusaur',
 '/wiki/Charmander',
 '/wiki/Charmeleon',
 '/wiki/Charizard',
 '/wiki/Squirtle',
 '/wiki/Wartortle',
 '/wiki/Blastoise',
 '/wiki/Caterpie',
 '/wiki/Metapod',
 '/wiki/Butterfree',
 '/wiki/Weedle',
 '/wiki/Kakuna',
 '/wiki/Beedrill',
 '/wiki/Pidgey',
 '/wiki/Pidgeotto',
 '/wiki/Pidgeot',
 '/wiki/Rattata',
 '/wiki/Raticate',
 '/wiki/Spearow',
 '/wiki/Fearow',
 '/wiki/Ekans',
 '/wiki/Arbok',
 '/wiki/Pikachu',
 '/wiki/Raichu',
 '/wiki/Sandshrew',
 '/wiki/Sandslash',
 '/wiki/Nidoran%E2%99%80',
 '/wiki/Nidorina',
 '/wiki/Nidoqueen',
 '/wiki/Nidoran%E2%99%82',
 '/wiki/Nidorino',
 '/wiki/Nidoking',
 '/wiki/Clefairy',
 '/wiki/Clefable',
 '/wiki/Vulpix',
 '/wiki/Ninetales',
 '/wiki/Jigglypuff',
 '/wiki/Wigglytuff',
 '/wiki/Zubat',
 '/wiki/Golbat',
 '/wiki/Oddish',
 '/wiki/Gloom',
 '/wiki/Vileplume',
 '/wiki/Paras',
 '/wiki/Parasect',
 '/wiki/Venonat',
 '/wiki/Venomoth',
 '/wiki/Diglett',
 '/wiki/Dugtrio',
 '/wiki/Meowth',
 '/wiki/Persian',


**根据 `first_level_links` 的链接，逐一爬取每个 Pokemon 的详细网页中的图片链接**

In [ ]:
img_urls = []
url_prefix = 'https://'

for i in range(10):
    soup3 = make_soup(first_level_links[i])
    l = soup3.find('table', class_='roundy bgwhite fulltable').find('img').get('data-srcset').split(',')[0]
    u = url_prefix + re.findall(r'media.*png', l)[0]
    img_links.append(u)
    
img_links

In [ ]:
# 创建文件夹路径
path = './pokemon-img/'
if not os.path.exists(path):
    os.makedirs(path)
else:
    print('文件路径已存在！')

In [ ]:
file_name = img_links[0].split('/')[-1]
file_name

In [ ]:
def download_img(path, img_link, index):
    '''
    根据 img_link 下载 Pokemon 图片
    
    params:
        path: 文件保存的文件夹路径
        img_link: 需下载的图片链接
        index: 用于遍历下载链接的坐标
    '''
    r = requests.get(img_link, stream=True)
    
    # 保存的文件名
    file_name = img_links[index].split('/')[-1]
    
    # 文件路径
    file_path = path+file_name
    
    # 需指定文件名，否则会出现 [Errno 21] Is a directory 错误
    with open(file_path, 'wb') as f:
        for chunk in r.iter_content(chunk_size=32):
            f.write(chunk)
        print('下载 {} 成功！'.format(file_name))

In [ ]:
for i in range(10):
    download_img(path, img_links[i], i)